In [87]:
# main reader
import numpy as np 
from matplotlib import pyplot 
import pandas as pd 

trainset={'x':[],'y':[]}
valset = {'x':[],'y':[]}

files={}
files['airq'] = ['aiqQuality_201804.csv','airQuality_201701-201801.csv','airQuality_201802-201803.csv']
files['grid'] = ['gridWeather_201804.csv','gridWeather_201701-201803.csv']
files['weather'] = ['observedWeather_201804.csv','observedWeather_201701-201801.csv','observedWeather_201802-201803.csv']
map_air = {'stationId': 'station_id', 'utc_time': 'time', 'PM2.5': 'PM25_Concentration', 'PM10': 'PM10_Concentration', 'NO2': 'NO2_Concentration', 'CO': 'CO_Concentration', 'O3': 'O3_Concentration', 'SO2': 'SO2_Concentration'}
map_grid = {'stationName': 'station_id', 'utc_time': 'time','wind_speed/kph':'wind_speed'}
map_weather = { 'utc_time': 'time'}


df1 = pd.read_csv('aiqQuality_201804.csv')
df2 = pd.read_csv('airQuality_201701-201801.csv').rename(columns=map_air)
df3 = pd.read_csv('airQuality_201802-201803.csv').rename(columns=map_air)
airq = pd.concat([df2, df3, df1], axis = 0) 

df1 = pd.read_csv('gridWeather_201804.csv')
df2 = pd.read_csv('gridWeather_201701-201803.csv').rename(columns=map_grid)
grid =  pd.concat([df2, df1], axis = 0) 

df1 = pd.read_csv('observedWeather_201802-201803.csv').rename(columns=map_weather)
df2 = pd.read_csv('observedWeather_201701-201801.csv').rename(columns=map_weather)
df3 = pd.read_csv('observedWeather_201804.csv')
observe = pd.concat([df1,df2,df3],axis = 0)


# the test for 201804

In [1]:
import preprocess
import numpy as np 
from matplotlib import pyplot 
import pandas as pd 
airq= pd.read_csv('aiqQuality_201804.csv')
observe = pd.read_csv('observedWeather_201804.csv')
grid= pd.read_csv('gridWeather_201804.csv')
after_data = preprocess.preprocess(airq,observe,grid)

Level 1 #null before 347
After:  102
Level 2 #null before 102
After:  66
Level 3 #null before 66
After:  48
Level 4 #null before 48
After:  30


C:\Users\Wangzhc\Desktop\train\preprocess.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[prefix+'_mean'] = rolling_obj.mean()
C:\Users\Wangzhc\Desktop\train\preprocess.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[prefix+'_median'] = rolling_obj.median()
C:\Users\Wangzhc\Desktop\train\preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [45]:
from sklearn.preprocessing import LabelEncoder
after_data['weather'] = LabelEncoder().fit_transform(after_data[['weather']])

C:\Users\Wangzhc\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [110]:
# get dataset for each station
# only take the valid set from dataset, which means it should contain both t time attribute & t+1 time label

dels = ['CO_Concentration','NO2_Concentration','O3_Concentration','PM10_Concentration','PM25_Concentration','SO2_Concentration','time','id','station_id']
def get_dataset(df,groupKey,attr,dele = True):
    grouped = df.groupby(groupKey)
    dataset ={}
    for key,group in grouped:
        attrs = []
        labels = []
        X = group[attr].values
        group = group.sort_values(by='time')
        group = group.reset_index(drop = True)
        #print(type(group))
        bucket = group.copy(deep=True)
        for d in dels:
            del bucket[d]
        #print(bucket.columns)
        for i in range(1,X.shape[0]):
            if pd.to_datetime(group['time'].loc[i]) - pd.to_datetime(group['time'].loc[i-1])==pd.Timedelta(hours=1): #pd.to_datetime(time_df['END_TIME']) - pd.to_datetime(time_df['START_TIME']):
                tmp = bucket.loc[i-1,:]
                if pd.isna(tmp.values).any():
                    continue
                attrs.append(tmp.values)
                labels.append(X[i])
            else:
                continue
        attrs = np.array(attrs).reshape(-1,attrs[0].shape[0])
        dataset[key] = (attrs,np.array(labels).reshape(-1))
         
    return dataset



In [113]:
# get dataset for different target and stations
# total 3*35 (x,y) tubles
target = ['PM25_Concentration','O3_Concentration','PM10_Concentration']
dataset = {}
for i in target:
    dataset[i] = get_dataset(after_data,'station_id',i)



In [ ]:
from lightgbm.sklearn import LGBMRegressor as lgbm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV as gdc
import evaluation


param = {"max_depth": [25,50, 75],
        "learning_rate" : [0.01,0.05,0.1],
        "num_leaves": [300,900,1200],
        "n_estimators": [200]
    }


stations = list(after_data['station_id'].unique())
for i in stations:
    print(i)
    preds = []
    labels = []
    for j in dataset.keys():
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        x_train, x_test, y_train, y_test = train_test_split(scaler.fit_transform(dataset[j][i][0]), dataset[j][i][1], test_size=0.2, random_state=42)
        search = gdc(lgbm(),param_grid= param,scoring='neg_mean_squared_error')
        search.fit(x_train,y_train)
        f = open('params/'+i+'_'+j+ '.txt','w')
        f.write(str(search.best_params_))
        f.close()
        result = search.predict(x_test).reshape(1,-1)
        preds.append(result)
        labels.append(y_test.reshape(-1,1))
        print(evaluation.smape(result,y_test))
        
    # one problem here is that the row num for each target is not the same, but here is just a validation
    
    #score = evaluation.smape(np.concatenate(whole,axis=1),np.concatenate(labels,axis=1))
    #print(score)

aotizhongxin_aq
0.2920352716792104
0.2727559232656714
0.24585497254946556
badaling_aq
0.25230856067791213
0.3363060077938236
0.44306737204380336
beibuxinqu_aq
0.23050516065293922
0.21844576037851465
0.3704998113098161
daxing_aq
0.2440218779715578
0.448812669304619
0.22684118997508815
dingling_aq
0.2593402617143255
0.1685204874553459
0.25876680856245093
donggaocun_aq
0.32413144166774854
0.13917630512442605
0.25295887160649905
dongsi_aq
0.2703766190580855
0.23438673910610114
0.24693243981643784
dongsihuan_aq
0.12021158862379694
0.5926449865927467
0.23683176819159932
fangshan_aq
0.21615417898808795
0.26619927436558866
0.2597828334049599
fengtaihuayuan_aq
0.30187382225133624
0.26291330756475634
0.223598741372646
guanyuan_aq
0.23118552658230868
0.2007986206516563
0.23270281253741484
gucheng_aq


array([0.6, 32.0, 202.0, 191.0, 126.0, 10.0, 2.0149762339197217, 17.0,
       2942786, 999.0532, 'aotizhongxin_aq', 27.0, '2018-04-01 09:00:00',
       4, 197.24, 15.81, 251.16666666666666, 262.0, 1507.0, 176.0, 279.0,
       1452.5666666666662, 234.5, 234.5, 469.0, 208.0, 261.0, 1404.5,
       149.5, 173.5, 897.0, 45.0, 214.0, 5017.9, 251.16666666666666,
       262.0, 1507.0, 176.0, 279.0, 1452.5666666666662, 234.5, 234.5,
       469.0, 208.0, 261.0, 1404.5, 149.5, 173.5, 897.0, 45.0, 214.0,
       5017.9, 251.16666666666666, 262.0, 1507.0, 176.0, 279.0,
       1452.5666666666662, 234.5, 234.5, 469.0, 208.0, 261.0, 1404.5,
       149.5, 173.5, 897.0, 45.0, 214.0, 5017.9, 251.16666666666666,
       262.0, 1507.0, 176.0, 279.0, 1452.5666666666662, 234.5, 234.5,
       469.0, 208.0, 261.0, 1404.5, 149.5, 173.5, 897.0, 45.0, 214.0,
       5017.9, 251.16666666666666, 262.0, 1507.0, 176.0, 279.0,
       1452.5666666666662, 234.5, 234.5, 469.0, 208.0, 261.0, 1404.5,
       149.5, 173.5, 897.

In [121]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((491, 97), (123, 97), (491,), (123,))

In [64]:
a = np.array([3,4,5]).reshape(-1,1)
b = np.array([3,4,5]).reshape(-1,1)
c = np.array([3,4,5]).reshape(-1,1)

In [68]:
np.append(a,b,axis=1)

array([[3, 3],
       [4, 4],
       [5, 5]])

In [72]:
np.concatenate([a,b,c],axis=1)

array([[3, 3, 3],
       [4, 4, 4],
       [5, 5, 5]])